## Create a Spark session

We have imported two libraries: **SparkSession** and **SQLContext**.

SparkSession is the entry point for programming Spark applications. It lets us interact with DataSet and DataFrame APIs provided by Spark. We set the application name by calling appName. The getOrCreate() method either returns a new SparkSession of the app or returns an existing one.

In [77]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

scSpark = SparkSession \
    .builder \
    .appName("reading_csv") \
    .getOrCreate()

## Read data from CSV (**Extract**)

Our next objective is to read CSV files. But before we move further, let’s play with some real data. For that purpose, we are using Supermarket’s sales data which I got from [Kaggle](https://www.kaggle.com/aungpyaeap/supermarket-sales). 

In [115]:
data_file = 'supermarket_sales.csv'
sdfData = scSpark.read.csv(data_file, header=True, sep=",").cache()

22/08/18 16:14:15 WARN CacheManager: Asked to cache already cached data.


In [116]:
sdfData.count()

1000

## Manipulating the Data (**Transform**)

### Running DataFrame methods on the Data

In [81]:
sdfData = sdfData.na.fill(value="NA")
sdfData = sdfData.withColumn(
    "Unit price after 1 year",
    sdfData["Unit price"]+((sdfData["Unit price"]/100)*6)
)
sdfData_filtered = sdfData.filter(
    sdfData["Customer type"] == 'Member'
    ).select([
        "Gender", 
        "City", 
        "Unit price", 
        "Quantity", 
        "Total"])
sdfData_filtered.show()


+------+---------+----------+--------+--------+
|Gender|     City|Unit price|Quantity|   Total|
+------+---------+----------+--------+--------+
|Female|   Yangon|     74.69|       7|548.9715|
|  Male|   Yangon|     58.22|       8| 489.048|
|Female|   Yangon|     68.84|       6| 433.692|
|Female|   Yangon|     36.26|       2|  76.146|
|Female| Mandalay|     54.84|       3| 172.746|
|Female| Mandalay|     14.48|       4|  60.816|
|  Male| Mandalay|     25.51|       4| 107.142|
|Female| Mandalay|     93.72|       6| 590.436|
|Female|   Yangon|     68.93|       7|506.6355|
|  Male|Naypyitaw|     86.04|       5|  451.71|
|  Male|   Yangon|     88.63|       3|279.1845|
|Female|   Yangon|     52.59|       8| 441.756|
|  Male| Mandalay|     78.07|       9|737.7615|
|Female|Naypyitaw|     99.42|       4| 417.564|
|Female|Naypyitaw|     68.12|       1|  71.526|
|  Male|   Yangon|     62.62|       5| 328.755|
|  Male| Mandalay|     30.12|       8| 253.008|
|Female| Mandalay|     86.72|       1|  

### Running SQL Queries on the Data

First, we create a temporary table out of the dataframe. For that purpose **registerTampTable** is used. In our case the table name is sales. Once it’s done you can use typical SQL queries on it.

In [82]:
sdfData_filtered.registerTempTable("sales")

/home/subhayu/.local/lib/python3.8/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [83]:
output = scSpark.sql('SELECT * from sales WHERE `Unit price` < 15 AND Quantity < 10')
output.show()

+------+---------+----------+--------+--------+
|Gender|     City|Unit price|Quantity|   Total|
+------+---------+----------+--------+--------+
|Female| Mandalay|     14.48|       4|  60.816|
|Female|Naypyitaw|     12.54|       1|  13.167|
|Female| Mandalay|     10.59|       3| 33.3585|
|Female| Mandalay|     13.22|       5|  69.405|
|Female|   Yangon|     14.23|       5| 74.7075|
|  Male|Naypyitaw|      14.7|       5|  77.175|
|  Male|Naypyitaw|     13.98|       1|  14.679|
|  Male|   Yangon|     10.13|       7| 74.4555|
|  Male|Naypyitaw|     12.05|       5| 63.2625|
|  Male|   Yangon|     11.94|       3|  37.611|
|Female|Naypyitaw|     10.53|       5| 55.2825|
|Female| Mandalay|     12.29|       9|116.1405|
|Female|   Yangon|     10.69|       5| 56.1225|
|Female|Naypyitaw|     10.16|       5|   53.34|
|  Male|Naypyitaw|     10.17|       1| 10.6785|
|Female| Mandalay|     11.85|       8|   99.54|
|Female|Naypyitaw|     14.87|       2|  31.227|
|  Male|   Yangon|     12.76|       2|  

In [84]:
output = scSpark.sql('SELECT COUNT(*) as total, City from sales GROUP BY City')
output.show()

+-----+---------+
|total|     City|
+-----+---------+
|  169|Naypyitaw|
|  165| Mandalay|
|  167|   Yangon|
+-----+---------+



## Saving the Output (**Load**)

Finally the load part of the ETL. What if we want to save this transformed data? Well, we have many options available, RDBMS, XML or JSON.

When we run the below code block, Sparks will create a folder with the name of the file, in our case it is filtered.json. Then, a file with the name _SUCCESS, which tells whether the operation was a success or not. In case it fails a file with the name _FAILURE is generated. Then, we find multiple files here. The reason for multiple files is that each work is involved in the operation of writing in the file.

In [85]:
output.write.save('filtered.json', 'json', 'overwrite')

If we want to create a single file(which is not recommended) then **coalesce** can be used that collects and reduces the data from all partitions to a single dataframe.

In [91]:
output.coalesce(3).write.format('json').save('filtered.json')